# GPT-2 Poetry Generator Project
Gholamreza Dar Feb 2023

## Install libraries

In [ ]:
!pip install transformers

## Imports

In [40]:
import os
import glob
import json
from pathlib import Path
import random
import time
import datetime

import pandas as pd
import numpy as np

import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelWithLMHead
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Config

from tqdm import tqdm
from IPython import display

## Download Dataset from huggingface

In [6]:
!wget -nc https://huggingface.co/datasets/Gholamreza/hafez/raw/main/train.csv
!wget -nc https://huggingface.co/datasets/Gholamreza/hafez/raw/main/test.csv

--2023-02-25 12:13:42--  https://huggingface.co/datasets/Gholamreza/hafez/raw/main/train.csv
Resolving huggingface.co (huggingface.co)... 3.231.67.228, 54.235.118.239, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|3.231.67.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 705449 (689K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 688.92K   617KB/s    in 1.1s    

2023-02-25 12:13:44 (617 KB/s) - ‘train.csv’ saved [705449/705449]

--2023-02-25 12:13:44--  https://huggingface.co/datasets/Gholamreza/hafez/raw/main/test.csv
Resolving huggingface.co (huggingface.co)... 54.235.118.239, 3.231.67.228, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|54.235.118.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81522 (80K) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]  79.61K   179K

In [7]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

## Select Model

In [8]:
model_name = "HooshvareLab/gpt2-fa"

## Tokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.add_special_tokens({
    "bos_token": '<bos>',
    "eos_token": '<eos>', 
    "pad_token": '<pad>',
    "unk_token": '<unk>'
})

2

In [12]:
print(tokenizer.encode("یه توپ دارم قلقلیه. حافظا مرد نکونام نمیرد هرگز"))
print(tokenizer.encode("<|startoftext|>")) # A special token that we are going to use to indicate start of mesra

[657, 4449, 3080, 1679, 299, 1075, 24, 6897, 268, 718, 33694, 15561, 275, 3776]
[6]


In [13]:
# Maximum length of a mesra
max_seq = 15

## Custom Dataset

In [14]:
from torch.utils.data import Dataset
torch.manual_seed(69)

# A custom dataset class
class PoemDataset(Dataset):
    def __init__(self, mesra_1_list, mesra_2_list, tokenizer, max_length=1024):

        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []

        for mesra_1, mesra_2 in zip(mesra_1_list, mesra_2_list):
            encodings_dict = tokenizer('' + mesra_1 + '<sep>' + mesra_2 + '',
                                       truncation=True,
                                       max_length=max_length,
                                       padding="max_length")

            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [15]:
# Generate a list of mesras (yeki darmioon!)
groups = train.groupby(by='v_position')
mesra_1_list, mesra_2_list = [], []
for grp, poems in groups:
    if grp == 0:
        mesra_1_list = list(poems['poem_text'])
    else:
        mesra_2_list = list(poems['poem_text'])

In [22]:
# A sample beyt
mesra_1_list[0], mesra_2_list[0]

('جز نقش تو در نظر نیامد ما را', 'جز کوی تو رهگذر نیامد ما را')

## Train Test Split

In [23]:
from torch.utils.data import random_split

dataset = PoemDataset(mesra_1_list, mesra_2_list, tokenizer, max_length=max_seq)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f"train size: {len(train_dataset)}")
print(f"validation size: {len(val_dataset)}")

train size: 3874
validation size: 431


## Dataloaders

In [24]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=8
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=8
)

## Create the model (gpt2-fa)

In [25]:
config = AutoConfig.from_pretrained(
    model_name,
    bos_token_id=tokenizer.encode('<bos>')[0], 
    eos_token_id=tokenizer.encode('<eos>')[0], 
    pad_token_id=tokenizer.encode('<pad>')[0],
    unk_token_id=tokenizer.encode('<unk>')[0],
)

### Download the weights

In [26]:
!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/pytorch_model.bin" -P /content/gpt2/
!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/config.json" -P /content/gpt2/

--2023-02-25 12:25:50--  https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 54.235.118.239, 3.231.67.228, 2600:1f18:147f:e850:e203:c458:10cd:fc3c, ...
Connecting to huggingface.co (huggingface.co)|54.235.118.239|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/HooshvareLab/gpt2-fa/46b0b806c740a0f0a9f056f5574c5fa896166fe844945fd3c849bf34365e5060?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1677587152&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL0hvb3NodmFyZUxhYi9ncHQyLWZhLzQ2YjBiODA2Yzc0MGEwZjBhOWYwNTZmNTU3NGM1ZmE4OTYxNjZmZTg0NDk0NWZkM2M4NDliZjM0MzY1ZTUwNjA%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVGhhbiI6eyJBV1M6RXBvY2hUa

### Download the config

In [27]:
!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/config.json" -P /content/gpt2/

--2023-02-25 12:25:56--  https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/config.json
Resolving huggingface.co (huggingface.co)... 54.235.118.239, 3.231.67.228, 2600:1f18:147f:e850:e203:c458:10cd:fc3c, ...
Connecting to huggingface.co (huggingface.co)|54.235.118.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 808 [text/plain]
Saving to: ‘/content/gpt2/config.json.1’

config.json.1       100%[===================>]     808  --.-KB/s    in 0s      

2023-02-25 12:25:57 (144 MB/s) - ‘/content/gpt2/config.json.1’ saved [808/808]



### Load the model weights

In [50]:
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np

# Loading the downloaded GPT2-fa model
configuration = GPT2Config.from_pretrained('./gpt2', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("./gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

# Move the model weights to gpu
device = torch.device("cuda")
model.cuda()

# random seed
seed_val = 69

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

### Hyperparams

In [51]:
epochs = 10
warmup_steps = 1e2
sample_every = 485

### Optimizer

In [52]:
from transformers import AdamW

# Optimizer
optimizer = AdamW(
    model.parameters(),
    lr=5e-4,
    eps=1e-8
)

In [53]:
from transformers import get_linear_schedule_with_warmup

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps)

In [54]:
mesra_1 = mesra_1_list[np.random.randint(0, len(mesra_1_list))]
sample_input = f"{mesra_1}<|startoftext|>"
print()
print("Sample input")
print(sample_input)

sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)

sample_outputs = model.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=1
)

gen_sample_output = tokenizer.decode(sample_outputs[0], skip_special_tokens=False)

print(f'Sample output: {gen_sample_output}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.



Sample input
ای دل گر از آن چاه زنخدان به درآیی<|startoftext|>
Sample output: ای دل گر از آن چاه زنخدان به درآیی<|startoftext|> می‌آید. «در آن چاه زنخدان‌ها و در آن چاه زنی، همهٔ کسانی که از دل آن چاه زنخشان می‌کنند، در آن چاه


## Training

In [55]:
# Utility function to format time (from pytorch examples)
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in tqdm(range(0, epochs), position=0):

    print(f'Epoch {epoch_i + 1} / {epochs}')

    # 1. Train Step
    t0 = time.time()
    total_train_loss = 0
    # Set the model in training mode
    model.train()
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), position=0):

        # Move to GPU
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        # Zero grad before backprop
        model.zero_grad()

        # Calculate the Loss
        outputs = model(b_input_ids, labels=b_labels, attention_mask=b_masks, token_type_ids=None)
        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Generate a  sample every epoch.
        if step == 0:
            elapsed = format_time(time.time() - t0)
            print()
            print(f'Batch {step} / {len(train_dataloader)}. Loss:{batch_loss}. Time:{elapsed}')

            # Set the model in eval mode
            model.eval()

            sample_mesra_1 = mesra_1_list[np.random.randint(0, len(mesra_1_list))]
            sample_input = f"{sample_mesra_1}<|startoftext|>"
            sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
            sample_input_ids = sample_input_ids.to(device)

            sample_outputs = model.generate(
                input_ids=sample_input_ids,
                do_sample=True,
                top_k=50,
                max_length=max_seq,
                top_p=0.95,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id
            )
            
            for i, output in enumerate(sample_outputs):
                o = tokenizer.decode(output, skip_special_tokens=False)
                o = o.replace("<|startoftext|>", "    ")

            print()
            print(f'Sample output: {o}')
            print()

            # Set the model in training mode
            model.train()

        # Backpropagation
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure elapsed time
    training_time = format_time(time.time() - t0)

    print()
    print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')
    print()

    # 2. Evaluation Step
    t0 = time.time()
    model.eval()
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in tqdm(validation_dataloader, total=len(validation_dataloader), position=0):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs = model(b_input_ids, attention_mask=b_masks, labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print()
    print(f'Validation loss: {avg_val_loss}. Validation Time: {validation_time}')
    print()

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print(f'Total training took {format_time(time.time()-total_t0)}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1 / 10


  0%|          | 1/485 [00:00<01:13,  6.56it/s]


Batch 0 / 485. Loss:8.400616645812988. Time:0:00:00

Sample output: در ره عشق از آن سوی فنا صد خطر است     فیه، در



100%|██████████| 485/485 [00:43<00:00, 11.04it/s]


...


Batch 0 / 485. Loss:0.2045382559299469. Time:0:00:00

Sample output: چنین قفس نه سزای چو من خوش الحانیست    روم به گلشن رضوان



100%|██████████| 485/485 [00:41<00:00, 11.78it/s]



Average Training Loss: 0.2596408551808485. Epoch time: 0:00:41



100%|██████████| 10/10 [07:51<00:00, 47.13s/it]


Validation loss: 8.003160768085056. Validation Time: 0:00:01

Total training took 0:07:51


In [104]:
def generator(sample_mesra_1, max_length=128, num_return_sequences=3, debug=True):
    model.eval()
    prompt = f"{sample_mesra_1}<|startoftext|>"

    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    generated = generated.to(device)

    decoded_outputs = model.generate(
        generated,
        do_sample=True,
        top_k=50,
        max_length=20, 
        top_p=0.95,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id
    )

    # for i, output in enumerate(decoded_outputs):
    #     o = tokenizer.decode(output, skip_special_tokens=False)
    #     out = o.replace("<|startoftext|>", "\t\t")
    #     out = out.replace("<pad>", "")
    #     # For eval
    #     mesra_2 = o.split("<|startoftext|>")[-1]
    #     if debug:
    #       print(o)
    for i, output in enumerate(decoded_outputs):
        o = tokenizer.decode(output, skip_special_tokens=False)
        out = o.replace("<|startoftext|>", " ---> ")
        out = out.replace("<pad>", "")
        out = out.replace("<sep>", "")
        mesra_2 = o.split("<|startoftext|>")[-1]
        if debug:
            print(out)

    return mesra_2

### Complete one beyt

In [107]:
sample_mesra_1 = mesra_1_list[20]
generator(sample_mesra_1, max_length=max_seq, num_return_sequences=1);

نی دولت دنیا به ستم می‌ارزد ---> نی لذت مستی‌اش المبلکه در وی شر


### Multiple outputs for a single beyt

In [109]:
for i in range(10):
    generator(mesra_1_list[20], max_length=max_seq, num_return_sequences=1)
    print()

نی دولت دنیا به ستم می‌ارزد ---> نی لذت مستی‌اش المبلکه به مستی‌

نی دولت دنیا به ستم می‌ارزد ---> نی لذت مستی‌اش المبلکه از مکدر

نی دولت دنیا به ستم می‌ارزد ---> نی لذت مستی‌اش المبلکه به زور می

نی دولت دنیا به ستم می‌ارزد ---> نی لذت مستی‌اش المبلغ الطفیل جل

نی دولت دنیا به ستم می‌ارزد ---> نی لذت مستی‌اش المبلکه باده به

نی دولت دنیا به ستم می‌ارزد ---> نی لذت مستی‌اش المبلکه به من درویش

نی دولت دنیا به ستم می‌ارزد ---> نی لذت مستی‌اش المبلکه بات بکشم

نی دولت دنیا به ستم می‌ارزد ---> نی لذت مستی‌اش المبلای من علی رغم

نی دولت دنیا به ستم می‌ارزد ---> نی لذت مستی‌اش المبلکه به فسوسی

نی دولت دنیا به ستم می‌ارزد ---> نی لذت مستی‌اش المبلغ الطاقه یا



In [110]:
for i in range(10):
    generator(mesra_1_list[14], max_length=max_seq, num_return_sequences=1)
    print()

امشب ز غمت میان خون خواهم خفت ---> وز بستر عافیت در عافیت میان

امشب ز غمت میان خون خواهم خفت ---> وز بستر عافیت این شوخی که در او

امشب ز غمت میان خون خواهم خفت ---> وز بستر عافیت امن و فسون امن

امشب ز غمت میان خون خواهم خفت ---> وز بستر عافیت را چاره از بستر ع

امشب ز غمت میان خون خواهم خفت ---> وز بستر عافیت بسته‌ام در ع

امشب ز غمت میان خون خواهم خفت ---> وز بستر عافیت اشارت مگر خوش است

امشب ز غمت میان خون خواهم خفت ---> وز بستر عافیت درگه پیر مغان پن

امشب ز غمت میان خون خواهم خفت ---> وز بستر عافیت میسر نمی‌شود ن

امشب ز غمت میان خون خواهم خفت ---> وز بستر عافیت نخواهد شد در آتش در

امشب ز غمت میان خون خواهم خفت ---> وز بستر عافیت را نیست درخور خبر شود



In [111]:
for i in range(10):
    generator(mesra_1_list[7], max_length=max_seq, num_return_sequences=1)
    print()

پیداست از آن میان چو بربست کمر ---> تا من ز کمر چه می‌خواهمم خون

پیداست از آن میان چو بربست کمر ---> تا من ز کمر چه شود میلم چه شود

پیداست از آن میان چو بربست کمر ---> تا من ز کمر چه تدبیرم چه کنم در

پیداست از آن میان چو بربست کمر ---> تا من ز کمر چه شود میلم چه شود

پیداست از آن میان چو بربست کمر ---> تا من ز کمر چه تدبیرمدرخواهم

پیداست از آن میان چو بربست کمر ---> تا من ز کمر چه تدبیرم اندیشم

پیداست از آن میان چو بربست کمر ---> تا من ز کمر چه تدبیرمجویم چه

پیداست از آن میان چو بربست کمر ---> تا من ز کمر چه تدبیرم در کمر چه

پیداست از آن میان چو بربست کمر ---> تا من ز کمر چه تدبیرمدر رکاب

پیداست از آن میان چو بربست کمر ---> تا من ز کمر چه رانی کمر چه تدبیرم



## Evaluation

In [112]:
groups = test.groupby(by='v_position')
mesra_1_list_test, mesra_2_list_test = [], []
for grp, poem in groups:
    if grp == 0:
        mesra_1_list_test = list(poem['poem_text'])
    else:
        mesra_2_list_test = list(poem['poem_text'])

In [120]:
for _ in range(10):
    generator(mesra_1_list_test[8], max_length=max_seq, num_return_sequences=1)
    print()

نه طبق سپهر و آن قرصه‌ی ماه و خور که هست ---> پیش عنقا قیاس اساس
نه طبق سپهر و آن قرصه‌ی ماه و خور که هست ---> یا رب از بساط قر
نه طبق سپهر و آن قرصه‌ی ماه و خور که هست ---> هزار نکته‌ی این
نه طبق سپهر و آن قرصه‌ی ماه و خور که هست ---> از شاه و وزیر
نه طبق سپهر و آن قرصه‌ی ماه و خور که هست ---> باده‌ی تقوی بر
نه طبق سپهر و آن قرصه‌ی ماه و خور که هست ---> نشان پیر مغان باد را
نه طبق سپهر و آن قرصه‌ی ماه و خور که هست ---> هزار یوسفم او به
نه طبق سپهر و آن قرصه‌ی ماه و خور که هست ---> باده با قدح به
نه طبق سپهر و آن قرصه‌ی ماه و خور که هست ---> هر چه بر وفق
نه طبق سپهر و آن قرصه‌ی ماه و خور که هست ---> بنده‌ی زمین علم


In [122]:
for _ in range(10):
    generator(mesra_1_list_test[1], max_length=max_seq, num_return_sequences=1)
    print()

گفتم سخن تو، گفت حافظ گفتا ---> وین دفتر بی‌خبر ز خط و خال تو هزاران

گفتم سخن تو، گفت حافظ گفتا ---> گفتا که احوال ما مپرس کرده‌ایم

گفتم سخن تو، گفت حافظ گفتا ---> کان شکر لهجه‌ی خوش لهجه‌ی خوش لهجه

گفتم سخن تو، گفت حافظ گفتا ---> در حق من این سخن این سخن ز که به سمع

گفتم سخن تو، گفت حافظ گفتا ---> با این همه او گفت و بر من دل آن سخن

گفتم سخن تو، گفت حافظ گفتا ---> گفت این سخن عشق آن زبان، که تو گفت از

گفتم سخن تو، گفت حافظ گفتا ---> گفتا که بوسه تو رخ ماه رخ ماه است و

گفتم سخن تو، گفت حافظ گفتا ---> در دایره قسمت اوضاع تو این سخن ز گیسوی سیاه

گفتم سخن تو، گفت حافظ گفتا ---> در چین زلف تو خون به من کی زان لب می

گفتم سخن تو، گفت حافظ گفتا ---> کو محتسبی که احوال مرا‌اش در جست



In [121]:
for _ in range(10):
    generator(mesra_1_list_test[3], max_length=max_seq, num_return_sequences=1)
    print()

از دست جوانی‌ام چو بربود عنان ---> تا چو شود سرو در میکده خواب آلوده نمازی

از دست جوانی‌ام چو بربود عنان ---> از جام گل دگر پیر می بهنسیم

از دست جوانی‌ام چو بربود عنان ---> در دست جوانی‌اش ز تنمدر جهان

از دست جوانی‌ام چو بربود عنان ---> کای دل خوش نوا به وداعشوه‌اش

از دست جوانی‌ام چو بربود عنان ---> از جام وصل می‌ام چو گل دگر یاد

از دست جوانی‌ام چو بربود عنان ---> دست قدرت نگر که بر دو دیده خودشمید

از دست جوانی‌ام چو بربود عنان ---> ز جام وصل می لبت کامگار می ل

از دست جوانی‌ام چو بربود عنان ---> کای دل کار ما از آن درگیرد شیران

از دست جوانی‌ام چو بربود عنان ---> از دست خوبان کهنمدولت صحبت آن

از دست جوانی‌ام چو بربود عنان ---> از دست رفته بودش روزگار رخت به بویان



In [123]:
for _ in range(10):
    generator(mesra_1_list_test[11], max_length=max_seq, num_return_sequences=1)
    print()

هیچ مژگان دراز و عشوه‌ی جادو نکرد ---> هر که با چنان جگر به جان بود به

هیچ مژگان دراز و عشوه‌ی جادو نکرد ---> یا رب از خاطرش اندیشه‌ی خود دل

هیچ مژگان دراز و عشوه‌ی جادو نکرد ---> همه از مهره بگذر و نقش خیال است و

هیچ مژگان دراز و عشوه‌ی جادو نکرد ---> هر که این قدرم همه رنگ و خیال

هیچ مژگان دراز و عشوه‌ی جادو نکرد ---> هر که این قدر قلب بدین دست داد و

هیچ مژگان دراز و عشوه‌ی جادو نکرد ---> هر که اقرار بدین حسن کندسرتو

هیچ مژگان دراز و عشوه‌ی جادو نکرد ---> هر که دانسته رود صرفه‌ای نبرد صرفه

هیچ مژگان دراز و عشوه‌ی جادو نکرد ---> همه‌ی آن که رایت به دست

هیچ مژگان دراز و عشوه‌ی جادو نکرد ---> هیچراهی نیست کان چنان که دیده‌ی

هیچ مژگان دراز و عشوه‌ی جادو نکرد ---> وان که زهدفروشان را به تیغ کشید



In [124]:
for _ in range(10):
    generator(mesra_1_list_test[13], max_length=max_seq, num_return_sequences=1)
    print()

در سفالین کاسه‌ی رندان به خواری منگرید ---> زانکه کاسه‌ی تقویست نصاب

در سفالین کاسه‌ی رندان به خواری منگرید ---> یا رب ز ابروی تو در محراب کنند

در سفالین کاسه‌ی رندان به خواری منگرید ---> به خنده گفت که حافظ راست از دنی

در سفالین کاسه‌ی رندان به خواری منگرید ---> ز قعر سر غیب ز رندان به

در سفالین کاسه‌ی رندان به خواری منگرید ---> ای پسته حرام از آن میان

در سفالین کاسه‌ی رندان به خواری منگرید ---> به آمرزش راز دل و زرق ز

در سفالین کاسه‌ی رندان به خواری منگرید ---> تا حریفان به جز ساغر که منع

در سفالین کاسه‌ی رندان به خواری منگرید ---> ز گریه و غمزه‌یاران

در سفالین کاسه‌ی رندان به خواری منگرید ---> زان رو که را نیست بر او منت

در سفالین کاسه‌ی رندان به خواری منگرید ---> ز کاسه‌ی رندان به کف



### BLEU

In [114]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu_score(gt, pred):
    gt = [gt.split()]
    pred = pred.split()
    score = sentence_bleu(gt, pred, weights=(1, 0, 0, 0))
    return score

In [119]:
scores = []
outputs = []
for mesra_1_test, mesra_2_test in zip(mesra_1_list_test, mesra_2_list_test):
    output = generator(mesra_1_test, max_length=max_seq, num_return_sequences=1, debug=False)
    outputs.append(output)
    score = calculate_bleu_score(gt=mesra_2_test, pred=output)
    scores.append(score)
  
print("BLEU:", np.average(scores))

BLEU: 0.05666578335754274
